In [3]:
import pandas as pd
import os
import numpy as np
from sqlalchemy import create_engine, text

server = "DESKTOP-HV85ADA\\SERVERSQL22"
bancodadosorigem =  "OLIP" 
driver = "ODBC Driver 17 for SQL Server"

conexao_origem  = f"mssql+pyodbc://{server}/{bancodadosorigem}?driver={driver}&Trusted_Connection=yes"
engine_origem  = create_engine(conexao_origem) # pool_size=10, max_overflow=30

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {engine} bem-sucedida")
            return
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
testar_conexao_sqlalchemy(engine_origem) 

pasta = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Kaggle\OLIST"

def ler_csv_para_df(pasta):
    
    dataframes = {}

    for arquivo in os.listdir(pasta):
        if arquivo.endswith('.csv'):
            nome_df = f"df_{os.path.splitext(arquivo)[0]}"

            caminho_completo = os.path.join(pasta, arquivo)
            dataframes[nome_df] = pd.read_csv(caminho_completo)

    return dataframes

def enviar_para_banco(dataframes, engine):
    """Envia todos os DataFrames para o banco de dados como tabelas separadas"""
    for nome_df, df in dataframes.items():
        nome_tabela = nome_df.replace("df_", "").upper()  # Remove 'df_' e coloca em maiúsculas
        
        try:
            # Envia para o banco de dados
            df.to_sql(
                name=nome_tabela,
                con=engine,
                if_exists='replace',  # 'replace' para sobrescrever, 'append' para adicionar
                index=False,           # Não incluir o índice do DataFrame
            )
            print(f"Tabela {nome_tabela} criada com sucesso no banco de dados!")
        except Exception as e:
            print(f"Erro ao enviar tabela {nome_tabela} para o banco: {str(e)}")

dfs = ler_csv_para_df(pasta)
enviar_para_banco(dfs, engine_origem)

globals().update(dfs)

for nome_df, df in dfs.items():
    print(f"\n{nome_df}")



Query teste executada com sucesso, conexão com o banco de dados Engine(mssql+pyodbc://DESKTOP-HV85ADA\SERVERSQL22/OLIP?Trusted_Connection=yes&driver=ODBC+Driver+17+for+SQL+Server) bem-sucedida
Tabela OLIST_CUSTOMERS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_GEOLOCATION_DATASET criada com sucesso no banco de dados!
Tabela OLIST_ORDERS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_ORDER_ITEMS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_ORDER_PAYMENTS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_ORDER_REVIEWS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_PRODUCTS_DATASET criada com sucesso no banco de dados!
Tabela OLIST_SELLERS_DATASET criada com sucesso no banco de dados!
Tabela PRODUCT_CATEGORY_NAME_TRANSLATION criada com sucesso no banco de dados!

df_olist_customers_dataset

df_olist_geolocation_dataset

df_olist_orders_dataset

df_olist_order_items_dataset

df_olist_order_payments_dataset

df_olist_order_reviews